# 多元统计分析作业二（小组）

- 胡义磊 2020010801005
- 张逸斐 2020010801006
- 阮登科 2020010801008
- 蔡与望 2020010801024

导入包，预设参数。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.tight_layout()

## 疫情前后行业发展趋势

读取年度数据分行业增加值数据集，并清洗数据。

In [ ]:
industries = pd.read_excel(
    "data/分行业增加值.xls",
    skiprows=2,
    skipfooter=2,
    index_col=0,
).T / 10000
industries.drop(columns="国内生产总值(亿元)", inplace=True)
industries.rename(
    columns=lambda name: name.replace("增加值(亿元)", ""),
    index=lambda name: name.replace("年", ""),
    inplace=True
)
industries.index.rename("年份", inplace=True)
industries.sort_index(inplace=True)

### 探索性分析

绘制折线图。各行业总体呈上升趋势。

In [ ]:
sns.lineplot(industries, dashes=False)
plt.show()

绘制箱线图。各行业数量级差距不大，且量纲相同，数据可以不做预处理。

In [ ]:
sns.boxplot(industries, orient="h")
plt.show()

绘制热力图。各行业的相关程度均较高，其中“住宿和餐饮业”和“农林牧渔业”相关程度最低。

In [ ]:
sns.heatmap(industries.corr(), annot=True, cmap="coolwarm")
plt.show()

### 主成分分析

绘制碎石图。前 2 个主成分的累计贡献率约 99.8%；并且在第 2 个主成分后，折线变得平缓。

In [ ]:
pca = PCA().fit(industries)
contributions = pd.Series(
    pca.explained_variance_ratio_,
    name="贡献率",
    index=pd.Index([str(i) for i in range(1, pca.n_components_ + 1)], name="主成分"),
)
contribution_cumsum = pd.Series(
    pca.explained_variance_ratio_.cumsum(),
    name="累计贡献率",
    index=pd.Index([str(i) for i in range(1, pca.n_components_ + 1)], name="主成分"),
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.lineplot(contributions, ax=axes[0])
sns.lineplot(contribution_cumsum, ax=axes[1])
plt.show()

所以接下来只需要考虑前 2 个主成分。

In [ ]:
pca = PCA(n_components=2)
raw_scores = pca.fit_transform(industries)

分析主成分内涵：

- 第一主成分中，“工业”和“其它行业”的权重最高，反映了“工业”和“其它行业”的综合实力。
- 第二主成分中，“工业”的权重最高，反映了“工业”的实力。

In [ ]:
components = pd.DataFrame(
    pca.components_.T,
    columns=["主成分1", "主成分2"],
    index=industries.columns
)
sns.heatmap(components, annot=True, cmap="coolwarm")
plt.show()

计算每年的综合得分，绘制散点图。

In [ ]:
scores = pd.DataFrame(
    raw_scores,
    columns=["主成分1", "主成分2"],
    index=industries.index
)
sns.scatterplot(scores, x="主成分1", y="主成分2")
for label, row in scores.iterrows():
    plt.text(row["主成分1"], row["主成分2"] , " " + str(label))
plt.show()

“工业”和“其它行业”的综合实力在 10 年内稳步提升。在疫情前后，这两个主要行业结合起来看，也并没有受到太大的影响。

“工业”的实力稍有波动，在疫情初期（2020）实力持续下跌，但疫情次年（2021）开始回升。

从坐标的距离上还可以发现：2019 和 2020 较近，但 2020 和 2021 较远。这是符合常识的：2020 年是疫情爆发的初期，中国各个行业都受到了巨大的冲击，所以这两个行业的增长率较低；但是 2021 年我国逐渐建立常态化防控机制，经济开始恢复，这两个行业又得到了较大的提升。

### 因子分析

绘制碎石图。前 2 个因子的累计贡献率约 99%；并且在第 2 个因子后，折线变得平缓。

In [ ]:
pre_covid_industries = industries.loc[:"2019"]
fa = FactorAnalyzer(rotation="varimax").fit(pre_covid_industries)
contributions = pd.Series(
    fa.get_factor_variance()[1],
    name="贡献率",
    index=pd.Index([str(i) for i in range(1, fa.n_factors + 1)], name="因子"),
)
contribution_cumsum = pd.Series(
    fa.get_factor_variance()[2],
    name="累计贡献率",
    index=pd.Index([str(i) for i in range(1, fa.n_factors + 1)], name="因子"),
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.lineplot(contributions, ax=axes[0])
sns.lineplot(contribution_cumsum, ax=axes[1])
plt.show()

所以接下来只需要考虑 2 个因子。

In [ ]:
fa = FactorAnalyzer(n_factors=2, rotation="varimax")
raw_scores = fa.fit_transform(pre_covid_industries)

分析因子内涵：

- 因子 1 贡献率最高的是“工业”、“建筑业”和“房地产业”，可以称为“基建因子”。
- 因子 2 贡献率最高的是“金融业”、“农林牧渔业”和“批发和零售业”，可以称为“金融因子”。

In [ ]:
factors = pd.DataFrame(
    fa.loadings_,
    columns=["因子1", "因子2"],
    index=industries.columns
)
sns.heatmap(factors, annot=True, cmap="coolwarm")
plt.show()

计算每年的因子得分，绘制散点图。

In [ ]:
scores = pd.DataFrame(
    raw_scores,
    columns=["因子1", "因子2"],
    index=pre_covid_industries.index
)
sns.scatterplot(scores, x="因子1", y="因子2")
for label, row in scores.iterrows():
    plt.text(row["因子1"], row["因子2"] , " " + str(label))
plt.show()


对于疫情之后的行业数据，也同样进行因子分析。

In [ ]:
covid_industries = industries.loc["2019":]
fa = FactorAnalyzer(n_factors=3, rotation="varimax").fit(covid_industries)
contributions = pd.Series(
    fa.get_factor_variance()[1],
    name="贡献率",
    index=pd.Index([str(i) for i in range(1, fa.n_factors + 1)], name="因子"),
)
contribution_cumsum = pd.Series(
    fa.get_factor_variance()[2],
    name="累计贡献率",
    index=pd.Index([str(i) for i in range(1, fa.n_factors + 1)], name="因子"),
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.lineplot(contributions, ax=axes[0])
sns.lineplot(contribution_cumsum, ax=axes[1])
plt.show()

fa = FactorAnalyzer(n_factors=2, rotation="varimax")
raw_scores = fa.fit_transform(covid_industries)

factors = pd.DataFrame(
    fa.loadings_,
    columns=["因子1", "因子2"],
    index=industries.columns
)
sns.heatmap(factors, annot=True, cmap="coolwarm")
plt.show()

scores = pd.DataFrame(
    raw_scores,
    columns=["因子1", "因子2"],
    index=covid_industries.index
)
sns.scatterplot(scores, x="因子1", y="因子2")
for label, row in scores.iterrows():
    plt.text(row["因子1"], row["因子2"] , " " + str(label))
plt.show()

分析因子内涵：

- 因子 2 贡献率最高的是“交通运输、仓储和邮政业”和“住宿和餐饮业”，可以称为“民生因子”。
- 因子 1 贡献率最高的是除了因子 1 中行业的其它所有行业，可以称为“非民生因子”。

通过因子分析，我们可以看出疫情前后，行业发展的驱动力和重心发生了变化。

疫情前，行业发展的主要驱动力是基建和金融。“基建因子”的影响先减后增，在 2015 达到极小值。“金融因子”的影响在 2015 前迅速增长，之后在 2017 前略有回退，随后又马上回暖。

疫情后，行业发展的主要驱动力是民生。“民生因子”的影响先减后增，在 2020 受疫情打击后，2021 马上有大幅增长。同时，“非民生因子”的影响也在不断增长。

### 总结

我们采用主成分分析和因子分析两种手段，对于“疫情对各行业的影响”进行了逻辑细致的分析。主要结论如下：

1. 各行业总体发展趋势稳中向好，增加值稳定上涨。
2. 发展最为突出的行业是“工业”和“其它行业”。
3. 疫情后，行业发展重心从基建和金融转移到民生上来，“交通运输、仓储和邮政业”和“住宿和餐饮业”正在蓬勃发展。